# DS 4400 Final Project : Credit Card Fraud Detection

#### Emily Chen, Glen Damian Lim, Tara Sawhney

#### Dataset : https://www.kaggle.com/datasets/kartik2112/fraud-detection

#### ML models: Logistic Regression, Decision Trees, Feedforward Neural Networks, Recurrent Neural Networks

In [1]:
import os 
os.environ["SM_FRAMEWORK"] = "tf.keras"
import segmentation_models as sm

Segmentation Models: using `tf.keras` framework.


In [143]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from collections import Counter

# ML libraries
from sklearn import metrics
from sklearn.feature_selection import RFE
from sklearn.preprocessing import StandardScaler
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.class_weight import compute_class_weight

from tensorflow import keras
import segmentation_models as sm
# Neural Networks libraries
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from keras.optimizers import SGD, Adam
from keras.losses import BinaryCrossentropy
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM

In [173]:
df_train = pd.read_csv('fraudTrain.csv')
df_test = pd.read_csv('fraudTest.csv')
# df_train.drop_duplicates(inplace=True)
# df_test.drop_duplicates(inplace=True)

In [163]:
df_train

,Unnamed: 0,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,...,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long,is_fraud
0,0,2019-01-01 00:00:18,2703186189652095,"fraud_Rippin, Kub and Mann",misc_net,4.97,Jennifer,Banks,F,561 Perry Cove,...,36.0788,-81.1781,3495,"Psychologist, counselling",1988-03-09,0b242abb623afc578575680df30655b9,1325376018,36.011293,-82.048315,0
1,1,2019-01-01 00:00:44,630423337322,"fraud_Heller, Gutmann and Zieme",grocery_pos,107.23,Stephanie,Gill,F,43039 Riley Greens Suite 393,...,48.8878,-118.2105,149,Special educational needs teacher,1978-06-21,1f76529f8574734946361c461b024d99,1325376044,49.159047,-118.186462,0
2,2,2019-01-01 00:00:51,38859492057661,fraud_Lind-Buckridge,entertainment,220.11,Edward,Sanchez,M,594 White Dale Suite 530,...,42.1808,-112.2620,4154,Nature conservation officer,1962-01-19,a1a22d70485983eac12b5b88dad1cf95,1325376051,43.150704,-112.154481,0
3,3,2019-01-01 00:01:16,3534093764340240,"fraud_Kutch, Hermiston and Farrell",gas_transport,45.00,Jeremy,White,M,9443 Cynthia Court Apt. 038,...,46.2306,-112.1138,1939,Patent attorney,1967-01-12,6b849c168bdad6f867558c3793159a81,1325376076,47.034331,-112.561071,0
4,4,2019-01-01 00:03:06,375534208663984,fraud_Keeling-Crist,misc_pos,41.96,Tyler,Garcia,M,408 Bradley Rest,...,38.4207,-79.4629,99,Dance movement psychotherapist,1986-03-28,a41d7549acf90789359a9aa5346dcb46,1325376186,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1296670,1296670,2020-06-21 12:12:08,30263540414123,fraud_Reichel Inc,entertainment,15.56,Erik,Patterson,M,162 Jessica Row Apt. 072,...,37.7175,-112.4777,258,Geoscientist,1961-11-24,440b587732da4dc1a6395aba5fb41669,1371816728,36.841266,-111.690765,0
1296671,1296671,2020-06-21 12:12:19,6011149206456997,fraud_Abernathy and Sons,food_dining,51.70,Jeffrey,White,M,8617 Holmes Terrace Suite 651,...,39.2667,-77.5101,100,"Production assistant, television",1979-12-11,278000d2e0d2277d1de2f890067dcc0a,1371816739,38.906881,-78.246528,0
1296672,1296672,2020-06-21 12:12:32,3514865930894695,fraud_Stiedemann Ltd,food_dining,105.93,Christopher,Castaneda,M,1632 Cohen Drive Suite 639,...,32.9396,-105.8189,899,Naval architect,1967-08-30,483f52fe67fabef353d552c1e662974c,1371816752,33.619513,-105.130529,0
1296673,1296673,2020-06-21 12:13:36,2720012583106919,"fraud_Reinger, Weissnat and Strosin",food_dining,74.90,Joseph,Murray,M,42933 Ryan Underpass,...,43.3526,-102.5411,1126,Volunteer coordinator,1980-08-18,d667cdcbadaaed3da3f4020e83591c83,1371816816,42.788940,-103.241160,0


## Data pre-processing

In [174]:
def datetime_column(df, col_name: str, hour: bool =False, new_col_prefix: str =''):
    df[col_name] = pd.to_datetime(df[col_name])

    if hour:
        new_col = new_col_prefix + '_hour'
        df[new_col] = df[col_name].dt.hour
    df[new_col_prefix + '_weekday'] = df[col_name].dt.weekday
    df[new_col_prefix + '_month'] = df[col_name].dt.strftime("%m")
    df[new_col_prefix + '_year'] = df[col_name].dt.year

# deriving additonal columns from 'trans_date_trans_time' and 'dob' columns
# datetime_column(df_train, 'trans_date_trans_time', True, 'trans')
# datetime_column(df_test, 'trans_date_trans_time', True, 'trans')
# datetime_column(df_train, 'dob', new_col_prefix='dob')
# datetime_column(df_test, 'dob', new_col_prefix='dob')


# dropping irrelevant columns
# df_train.drop(['Unnamed: 0','cc_num','merchant', 'first', 'last','street','zip', 'dob', 'trans_num', 'trans_date_trans_time'], axis=1, inplace=True)
# df_test.drop(['Unnamed: 0','cc_num','merchant', 'first', 'last','street','zip', 'dob', 'trans_num', 'trans_date_trans_time'], axis=1, inplace=True)
df_train.drop(['gender','city','state','job','unix_time','Unnamed: 0','cc_num','merchant', 'first', 'last','street','zip', 'dob', 'trans_num', 'trans_date_trans_time'], axis=1, inplace=True)
df_test.drop(['gender','city','state','job','unix_time','Unnamed: 0','cc_num','merchant', 'first', 'last','street','zip', 'dob', 'trans_num', 'trans_date_trans_time'], axis=1, inplace=True)
# Convert categorical columns
# categorical_column_names = ['city', 'state', 'job', 'category']
# categorical_column_names = ['category']

# for cat_name in categorical_column_names:
#     df_train[cat_name] = pd.factorize(df_train[cat_name])[0]
#     df_test[cat_name] = pd.factorize(df_test[cat_name])[0]

In [175]:
df_train

,category,amt,lat,long,city_pop,merch_lat,merch_long,is_fraud
0,misc_net,4.97,36.0788,-81.1781,3495,36.011293,-82.048315,0
1,grocery_pos,107.23,48.8878,-118.2105,149,49.159047,-118.186462,0
2,entertainment,220.11,42.1808,-112.2620,4154,43.150704,-112.154481,0
3,gas_transport,45.00,46.2306,-112.1138,1939,47.034331,-112.561071,0
4,misc_pos,41.96,38.4207,-79.4629,99,38.674999,-78.632459,0
...,...,...,...,...,...,...,...,...
1296670,entertainment,15.56,37.7175,-112.4777,258,36.841266,-111.690765,0
1296671,food_dining,51.70,39.2667,-77.5101,100,38.906881,-78.246528,0
1296672,food_dining,105.93,32.9396,-105.8189,899,33.619513,-105.130529,0
1296673,food_dining,74.90,43.3526,-102.5411,1126,42.788940,-103.241160,0


#### Resampling methods 

In [176]:
class_count_0, class_count_1 = df_train['is_fraud'].value_counts()

class_0 = df_train[df_train['is_fraud'] == 0]
class_1 = df_train[df_train['is_fraud'] == 1]

class_0_under = class_0.sample(class_count_1)
df_undersampling = pd.concat([class_0_under, class_1], axis=0)
# undersampled_trainX = test_under.drop('is_fraud', axis =1)
# undersampled_trainy = test_under['is_fraud']

class_1_over = class_1.sample(class_count_0, replace=True)
df_oversampling = pd.concat([class_1_over, class_0], axis=0)
# oversampled_trainX = test_over.drop('is_fraud', axis =1)
# oversampled_trainy = test_over['is_fraud']

#### Feature Selection and Data Scaling

In [177]:
from sklearn.model_selection import train_test_split

def select_scale_features(model, n_features, df_train, df_test):
#     df_train = df_train.astype(int)
#     df_test = df_test.astype(int)
#     X_train = df_train.drop('is_fraud', axis=1)
#     y_train = df_train['is_fraud']
#     X_test = df_test.drop('is_fraud', axis=1)
#     y_test = df_test['is_fraud']

    # NOTE: df_train IS BASICALLY df_oversampling
    X_train, X_test, y_train, y_test = train_test_split(df_train.drop('is_fraud', axis=1),df_train['is_fraud'] , train_size=0.7, test_size=0.3, random_state=42)
    
    # Normalize the data
    mean = np.mean(X_train)
    std = np.std(X_train)
    X_train = (X_train - mean) / std
    X_test = (X_test - mean) / std
    
    if model != None:
        selector = RFE(model, n_features_to_select=n_features)
        selector.fit(X_train, y_train)
        X_train = X_train[X_train.columns[selector.support_]]
        X_test = X_test[X_test.columns[selector.support_]]
        

    return X_train, y_train, X_test, y_test

In [178]:
df_oversampling =pd.get_dummies(df_oversampling, drop_first=True)
# df_train = pd.get_dummies(df_train, drop_first=True)


X_train, y_train, X_test, y_test = select_scale_features(LogisticRegression(), 10, df_oversampling, df_test)

In [184]:
Counter(y_test)

Counter({1: 386399, 0: 387103})

## Models

#### Logistic Regression

In [181]:
def logistic_regression(X_train, y_train, X_test, y_test):
    clf = LogisticRegression().fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    print('accuracy:', metrics.accuracy_score(y_test, y_pred))
    print(metrics.classification_report(y_test,
                                    y_pred))
    return y_pred

In [182]:
y_pred = logistic_regression(X_train, y_train, X_test, y_test)

accuracy: 0.826728308394807
              precision    recall  f1-score   support

           0       0.79      0.90      0.84    387103
           1       0.88      0.76      0.81    386399

    accuracy                           0.83    773502
   macro avg       0.83      0.83      0.83    773502
weighted avg       0.83      0.83      0.83    773502



In [183]:
print(Counter(y_pred))
print(Counter(y_test))

Counter({0: 442157, 1: 331345})
Counter({0: 387103, 1: 386399})


#### Decision Trees

In [185]:
def decision_tree(X_train, y_train, X_test, y_test, criterion, max_depth, min_samples_split, random_state: int = 3000):
    clf = DecisionTreeClassifier(random_state=random_state, criterion = criterion, max_depth = max_depth,\
                                 min_samples_split = min_samples_split)
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    print('accuracy:', metrics.accuracy_score(y_test, y_pred))
    print(metrics.classification_report(y_test,
                                    y_pred))
    
    return y_pred

def random_forest(X_train, y_train, X_test, y_test, random_state: int = 3000):
    clf = RandomForestClassifier(random_state=random_state)
    clf.fit(X_train, y_train)
    
    y_pred = clf.predict(X_test)
    print('accuracy:', metrics.accuracy_score(y_test, y_pred))
    print(metrics.classification_report(y_test,
                                    y_pred))
    
    return y_pred

In [186]:
y_pred = decision_tree(X_train, y_train, X_test, y_test, "gini", 20, 10)

accuracy: 0.9702599346866588
              precision    recall  f1-score   support

           0       0.99      0.95      0.97    387103
           1       0.96      0.99      0.97    386399

    accuracy                           0.97    773502
   macro avg       0.97      0.97      0.97    773502
weighted avg       0.97      0.97      0.97    773502



In [ ]:
y_pred = random_forest(X_train, y_train, X_test, y_test)

In [ ]:
print(Counter(y_pred))
print(Counter(y_test))

#### Feedforward Neural Network

In [160]:
def ffnn(X_train, y_train, X_test,epochs, batch_size, weights, optimizer, loss, metrics):
    model = Sequential()
    # input/output dimensions
    # hidden layer -- same number of hidden units as above
    model.add(Dense(2048, activation='relu', input_shape = (X_train.shape[1],)))
    model.add(Dropout(.2))
    model.add(Dense(1024, activation='relu'))
    model.add(Dropout(.2))
    model.add(Dense(512, activation='relu'))
    # output layer
    model.add(Dropout(.2))
    model.add(Dense(1, activation='sigmoid'))
              
    # configure the learning process
    model.compile(loss=loss,
                  optimizer=optimizer,
                  metrics=metrics)
    weights = compute_class_weight(class_weight="balanced",classes=np.unique(y_train),y=y_train)
    class_weights = {0: weights[0], 1: weights[1]}
#  class_weight = class_weights
    model.fit(X_train, y_train, 
              epochs= epochs, batch_size = batch_size, verbose=1, validation_split = 0.3, class_weight = class_weights)
    print(model.output)
    y_pred = model.predict(X_test)
    return y_pred

In [161]:
model = Sequential()
preds = ffnn(X_train, y_train, X_test, 10, 128, 0, 'adam', 'binary_crossentropy', ['accuracy'])

Train on 907672 samples, validate on 389003 samples
Epoch 1/10
907672/907672 [==============================] - 54s 60us/step - loss: 0.3709 - accuracy: 0.9493 - val_loss: 0.2917 - val_accuracy: 0.9894
Epoch 2/10
907672/907672 [==============================] - 54s 59us/step - loss: 0.3621 - accuracy: 0.9784 - val_loss: 0.2903 - val_accuracy: 0.9860
Epoch 3/10
907672/907672 [==============================] - 54s 59us/step - loss: 0.3286 - accuracy: 0.9854 - val_loss: 0.2840 - val_accuracy: 0.9906
Epoch 4/10
907672/907672 [==============================] - 54s 60us/step - loss: 0.4177 - accuracy: 0.9871 - val_loss: 0.2969 - val_accuracy: 0.9897
Epoch 5/10
907672/907672 [==============================] - 55s 60us/step - loss: 0.3788 - accuracy: 0.9857 - val_loss: 0.2772 - val_accuracy: 0.9914
Epoch 6/10
907672/907672 [==============================] - 55s 61us/step - loss: 0.3688 - accuracy: 0.9862 - val_loss: 0.2909 - val_accuracy: 0.9928
Epoch 7/10
907672/907672 [======================

KeyboardInterrupt: 

In [126]:
t = [1 if pred < 0.5 else 0 for pred in preds]

In [127]:
a = metrics.classification_report(y_test, t)
print(a)

              precision    recall  f1-score   support

           0       0.93      0.05      0.10    553574
           1       0.00      0.06      0.00      2145

    accuracy                           0.05    555719
   macro avg       0.47      0.06      0.05    555719
weighted avg       0.93      0.05      0.10    555719



In [ ]:
print(Counter(t))
print(Counter(y_test))

In [ ]:
len(y_test)

In [ ]:
t = [1 if pred < 0.5 else 0 for pred in preds]


In [ ]:
Counter(t)

#### Recurrent Neural Network

In [ ]:
Counter(y_train)

In [ ]:
np.around(preds)